<a href="https://colab.research.google.com/github/anupa-aa/DeepLearning.AI-Courses/blob/master/Improving_MNIST_with_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3: Improve MNIST with Convolutions

In the videos you looked at how you would improve Fashion MNIST using Convolutions. For this exercise see if you can improve MNIST to 99.5% accuracy or more by adding only a single convolutional layer and a single MaxPooling 2D layer to the model from the  assignment of the previous week.

You should stop training once the accuracy goes above this amount. It should happen in less than 10 epochs, so it's ok to hard code the number of epochs for training, but your training must end once it hits the above metric. If it doesn't, then you'll need to redesign your callback.

When 99.5% accuracy has been hit, you should print out the string "Reached 99.5% accuracy so cancelling training!"


## Importing modules

In [1]:
# grader-required-cell

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Loading the data

In [21]:
# Get current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "/mnist.npz")

# Get only training set
(training_images, training_labels), (validation_images, validation_labels) = tf.keras.datasets.mnist.load_data(path=data_path)


## Pre-processing the data

One important step when dealing with image data is to preprocess the data. During the preprocess step you can apply transformations to the dataset that will be fed into your convolutional neural network.

Here you will apply two transformations to the data:
- Reshape the data so that it has an extra dimension. The reason for this
is that commonly you will use 3-dimensional arrays (without counting the batch dimension) to represent image data. The third dimension represents the color using RGB values. This data might be in black and white format so the third dimension doesn't really add any additional information for the classification process but it is a good practice regardless.


- Normalize the pixel values so that these are values between 0 and 1. You can achieve this by dividing every value in the array by the maximum.

Remember that these tensors are of type `numpy.ndarray` so you can use functions like [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) or [divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) to complete the `reshape_and_normalize` function below:

In [22]:
# grader-required-cell

# GRADED FUNCTION: reshape_and_normalize

def reshape_and_normalize(images):

    ### START CODE HERE

    # Reshape the images to add an extra dimension
    images = images.reshape(-1, 28, 28, 1)

    # Normalize pixel values
    images = images/255

    ### END CODE HERE

    return images

In [23]:
# grader-required-cell

# Reload the images in case you run this cell multiple times
(training_images, _), (validation_images, _) = tf.keras.datasets.mnist.load_data(path=data_path)

# Apply your function
training_images = reshape_and_normalize(training_images)
validation_images = reshape_and_normalize(validation_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


## Defining your callback

Now complete the callback that will ensure that training will stop after an accuracy of 99.5% is reached.

Define your callback in such a way that it checks for the metric `accuracy` (`acc` can normally be used as well but the grader expects this metric to be called `accuracy` so to avoid getting grading errors define it using the full word).

In [27]:
# Remember to inherit from the correct class
class myCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):
        if logs.get("val_accuracy") is not None and logs.get("val_accuracy") > 0.99:
            print("Reached 99% accuracy so stopped training")
            self.model.stop_training = True
    # Define the method that checks the accuracy at the end of each epoch

## Convolutional Model

Finally, complete the `convolutional_model` function below. This function should return your convolutional neural network.

**Your model should achieve an accuracy of 99.5% or more before 10 epochs to pass this assignment.**

**Hints:**
- You can try any architecture for the network but try to keep in mind you don't need a complex one. For instance, only one convolutional layer is needed.
- In case you need extra help you can check out an architecture that works pretty well at the end of this notebook.
- To avoid timeout issues with the autograder, please limit the number of units in your convolutional and dense layers. An exception will be raised if your model is too large.

In [29]:
# grader-required-cell

# GRADED FUNCTION: convolutional_model
def deep_convolutional_model():
    ### START CODE HERE

    # Define the model
    model = tf.keras.models.Sequential([
        keras.layers.Conv2D(16, (3,3), activation="relu", input_shape=(28,28,1)),
        keras.layers.MaxPooling2D(2,2),
        keras.layers.Conv2D(32, (3,3), activation="relu"),
        keras.layers.MaxPooling2D(2,2),
        keras.layers.Conv2D(32, (3,3), activation="relu"),

        keras.layers.Flatten(),
        keras.layers.Dense(64,activation="relu"),
        keras.layers.Dense(10, activation="softmax")






    ])
    ### END CODE HERE

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [25]:
# grader-required-cell

# GRADED FUNCTION: convolutional_model
def convolutional_model():
    ### THIS ONE IS BETTER THAN THE ONE ABOVE!

    # Define the model
    model = tf.keras.models.Sequential([
        keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
        keras.layers.MaxPooling2D(2,2),

        keras.layers.Flatten(),
        keras.layers.Dense(128,activation="relu"),
        keras.layers.Dense(10, activation="softmax")






    ])
    ### END CODE HERE

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

## Train model 1

In [28]:
# Save your untrained model
model = convolutional_model()

# Get number of weights
model_params = model.count_params()

# Unit test to limit the size of the model
assert model_params < 1000000, (
    f'Your model has {model_params:,} params. For successful grading, please keep it '
    f'under 1,000,000 by reducing the number of units in your Conv2D and/or Dense layers.'
)

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=10,
                    validation_data=(validation_images, validation_labels),
                    callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 8s 3ms/step - loss: 0.1493 - accuracy: 0.9561 - val_loss: 0.0616 - val_accuracy: 0.9796
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0523 - accuracy: 0.9838 - val_loss: 0.0480 - val_accuracy: 0.9846
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0344 - accuracy: 0.9890 - val_loss: 0.0409 - val_accuracy: 0.9866
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0221 - accuracy: 0.9927 - val_loss: 0.0486 - val_accuracy: 0.9848
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0150 - accuracy: 0.9953 - val_loss: 0.0495 - val_accuracy: 0.9862
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0118 - accuracy: 0.9962 - val_loss: 0.0447 - val_accuracy: 0.9870
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0085 - accuracy: 0.9974 - val_loss: 0.0454 - val_accuracy:

## Train model 2

In [30]:
# Save your untrained model
model = deep_convolutional_model()

# Get number of weights
model_params = model.count_params()

# Unit test to limit the size of the model
assert model_params < 1000000, (
    f'Your model has {model_params:,} params. For successful grading, please keep it '
    f'under 1,000,000 by reducing the number of units in your Conv2D and/or Dense layers.'
)

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=10,
                    validation_data=(validation_images, validation_labels),
                    callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 41s 5ms/step - loss: 0.1724 - accuracy: 0.9471 - val_loss: 0.0512 - val_accuracy: 0.9841
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0556 - accuracy: 0.9830 - val_loss: 0.0434 - val_accuracy: 0.9869
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0412 - accuracy: 0.9867 - val_loss: 0.0340 - val_accuracy: 0.9895
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0306 - accuracy: 0.9903 - val_loss: 0.0314 - val_accuracy: 0.9893
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0258 - accuracy: 0.9920 - val_loss: 0.0273 - val_accuracy: 0.9912
